In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import boxcox 
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge, Lasso
from scipy.special import inv_boxcox
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")
import plotly.express as px
from plotly.offline import init_notebook_mode

import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy as np
import os
from datetime import datetime
import holidays
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
us_holidays=holidays.US()


In [45]:
pip install holidays

  Created wheel for pymeeus: filename=PyMeeus-0.5.11-py3-none-any.whl size=730973 sha256=3b10a1e752d07d54221230c8d7fd9f34a4f8e1854cd3b86872b2d5320aca518d
  Stored in directory: c:\users\student\appdata\local\pip\cache\wheels\a0\8b\b2\810ae5a6f970c8be4725353400d643c90de1c0f023a9884ee7
Successfully built pymeeus
Note: you may need to restart the kernel to use updated packages.


In [4]:
df=pd.read_csv('C:/Users/student/documents/wait_project/alien_saucers.csv')

In [5]:
df.head()

,date,datetime,SACTMIN,SPOSTMIN
0,6/30/2018,6/30/2018 7:44,NaN,150.0
1,6/30/2018,6/30/2018 7:51,NaN,150.0
2,6/30/2018,6/30/2018 7:58,NaN,150.0
3,6/30/2018,6/30/2018 8:02,NaN,150.0
4,6/30/2018,6/30/2018 8:07,NaN,150.0


In [7]:
df_meta=pd.read_csv('C:/Users/student/documents/wait_project/metadata (1).csv')

In [8]:
df_meta.head()

,DATE,WDW_TICKET_SEASON,DAYOFWEEK,DAYOFYEAR,WEEKOFYEAR,MONTHOFYEAR,YEAR,SEASON,HOLIDAYPX,HOLIDAYM,...,HSFIREWKS,AKPRDDAY,AKPRDDT1,AKPRDDT2,AKPRDDN,AKFIREN,AKSHWNGT,AKSHWNT1,AKSHWNT2,AKSHWNN
0,1/1/2015,NaN,5,0,0,1,2015,CHRISTMAS PEAK,0,5,...,1,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
1,1/2/2015,NaN,6,1,0,1,2015,CHRISTMAS,2,5,...,1,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
2,1/3/2015,NaN,7,2,0,1,2015,CHRISTMAS,3,0,...,1,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
3,1/4/2015,NaN,1,3,1,1,2015,CHRISTMAS,4,0,...,1,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
4,1/5/2015,NaN,2,4,1,1,2015,CHRISTMAS,5,0,...,1,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN


In [9]:
df2=df.drop(columns='SPOSTMIN')

In [10]:
df2.count()

date        129876
datetime    129876
SACTMIN       3031
dtype: int64

In [19]:
df3 = df2.dropna()

In [12]:
df3.count()

date        3031
datetime    3031
SACTMIN     3031
dtype: int64

In [13]:
df3.head()

,date,datetime,SACTMIN
79,6/30/2018,6/30/2018 15:54,37.0
87,6/30/2018,6/30/2018 16:29,49.0
141,6/30/2018,6/30/2018 22:05,50.0
142,6/30/2018,6/30/2018 22:05,53.0
169,7/1/2018,7/1/2018 7:16,27.0


In [34]:
df3=df3.rename(columns={'date':'DATE'})

In [20]:
df3.count()

date        3031
datetime    3031
SACTMIN     3031
dtype: int64

In [15]:
df_merge=df3.merge(df_meta, on='DATE')

In [21]:
df_merge.count()

DATE                 2178
datetime             2178
SACTMIN              2178
WDW_TICKET_SEASON    1119
DAYOFWEEK            2178
                     ... 
AKFIREN                 0
AKSHWNGT             2178
AKSHWNT1             1675
AKSHWNT2             1184
AKSHWNN              1675
Length: 183, dtype: int64

In [27]:
df_merge.count()

DATE                 2178
datetime             2178
SACTMIN              2178
WDW_TICKET_SEASON    1119
DAYOFWEEK            2178
                     ... 
AKFIREN                 0
AKSHWNGT             2178
AKSHWNT1             1675
AKSHWNT2             1184
AKSHWNN              1675
Length: 183, dtype: int64

In [17]:
df_merge.value_counts()

Series([], dtype: int64)

In [39]:
df_merge.head()

,DATE,datetime,SACTMIN,WDW_TICKET_SEASON,DAYOFWEEK,DAYOFYEAR,WEEKOFYEAR,MONTHOFYEAR,YEAR,SEASON,...,AKPRDDAY,AKPRDDT1,AKPRDDT2,AKPRDDN,AKFIREN,AKSHWNGT,AKSHWNT1,AKSHWNT2,AKSHWNN,date
0,6/30/2018,2018-06-30 07:44:00,37.0,peak,7,180,25,6,2018,SPRING,...,0,NaN,NaN,NaN,NaN,2,21:15,22:15,Rivers of Light,2018-06-30
1,6/30/2018,2018-06-30 07:51:00,49.0,peak,7,180,25,6,2018,SPRING,...,0,NaN,NaN,NaN,NaN,2,21:15,22:15,Rivers of Light,2018-06-30
2,6/30/2018,2018-06-30 07:58:00,50.0,peak,7,180,25,6,2018,SPRING,...,0,NaN,NaN,NaN,NaN,2,21:15,22:15,Rivers of Light,2018-06-30
3,6/30/2018,2018-06-30 08:02:00,53.0,peak,7,180,25,6,2018,SPRING,...,0,NaN,NaN,NaN,NaN,2,21:15,22:15,Rivers of Light,2018-06-30
4,7/1/2018,2018-06-30 08:07:00,27.0,peak,1,181,26,7,2018,SUMMER BREAK,...,0,NaN,NaN,NaN,NaN,2,21:15,22:15,Rivers of Light,2018-06-30


In [56]:
df_merge['DATE'] = pd.to_datetime(df_merge['DATE'])
df_merge['datetime'] = pd.to_datetime(df_merge['datetime'])

df_merge['time_pct'] = df_merge.datetime.apply(lambda time: (time.hour * 60 + time.minute) / (24 * 60))

L = ['year', 'month', 'day', 'dayofweek', 'dayofyear', 'weekofyear', 'quarter','is_month_start','is_month_end','is_quarter_start']
df_merge = df_merge.join(pd.concat((getattr(df_merge['date'].dt, i).rename(i) for i in L), axis=1))

# Get the holiday names
def k(x):
    if x['Holiday']:
        return us_holidays.get(x['DATE'])
    else:
        return "NoHoliday"


ValueError: columns overlap but no suffix specified: Index(['year', 'month', 'day', 'dayofweek', 'dayofyear', 'weekofyear',
       'quarter', 'is_month_start', 'is_month_end', 'is_quarter_start'],
      dtype='object')

In [57]:
df_merge.tail()

,DATE,datetime,SACTMIN,WDW_TICKET_SEASON,DAYOFWEEK,DAYOFYEAR,WEEKOFYEAR,MONTHOFYEAR,YEAR,SEASON,...,year,month,day,dayofweek,dayofyear,weekofyear,quarter,is_month_start,is_month_end,is_quarter_start
2173,2021-08-29,2018-07-13 10:51:00,24.0,NaN,1,240,35,8,2021,NaN,...,2018,7,13,4,194,28,3,False,False,False
2174,2021-08-29,2018-07-13 10:58:00,27.0,NaN,1,240,35,8,2021,NaN,...,2018,7,13,4,194,28,3,False,False,False
2175,2021-08-29,2018-07-13 11:02:00,3.0,NaN,1,240,35,8,2021,NaN,...,2018,7,13,4,194,28,3,False,False,False
2176,2021-08-31,2018-07-13 11:09:00,10.0,NaN,3,242,35,8,2021,NaN,...,2018,7,13,4,194,28,3,False,False,False
2177,2021-08-31,2018-07-13 11:16:00,15.0,NaN,3,242,35,8,2021,NaN,...,2018,7,13,4,194,28,3,False,False,False
